# Information Retrieval with TF-IDF
This notebook implements an information retrieval system using TF-IDF, cosine similarity, and inverted index techniques.

In [1]:
import pandas as pd
from math import log, sqrt

## Load Dataset

In [2]:
def load_dataset(file_path):
    dataset = pd.read_excel(file_path)
    return dataset['Abstrak'].tolist(), dataset['Judul Jurnal'].tolist(), dataset

# Load dataset
file_path = r'C:\\Users\\Monika Hermiani\\Documents\\Semester 7\\Information Retrieval\\Dataset Jurnal\\dataset IR.xlsx'
abstracts, titles, dataset = load_dataset(file_path)
print(f'Loaded {len(abstracts)} documents.')
print(dataset.head())


Loaded 300 documents.
                                        Judul Jurnal  \
0  Akurasi Klasifikasi Citra Digital Scenes RGB M...   
1  Akurasi Text Mining Menggunakan Algoritma K-Ne...   
2  Alat Pendeteksi Ekspresi Wajah pada Pengendara...   
3  ALGORITMA DECISION TREE (C4.5) UNTUK MEMPREDIK...   
4  ALGORITMA NAÏVE BAYES DALAM ANALISIS SENTIMEN...   

                                             Abstrak  \
0  Abstrak— Gambar citra digital merupakan larik ...   
1  Abstrak – Sejak tahun 2004, LKBN ANTARA masih ...   
2  Abstrak.\nSuatu gerakan atau posisi otot wajah...   
3  Abstrak – Pelaksanaan evaluasi \nkepuasan maha...   
4  ABSTRAK Kebiasaan masyarakat untuk mem-posting...   

                                     Penulis Tahun Terbit  \
0            Elza Fitriana Saraswita, Sukemi         2019   
1                        Windu Gata, Purnomo         2016   
2         Fadila Denta Sukma, Riki Mukhaiyar         2022   
3             Ade Yuliana, Duwi Bayu Pratomo         2017   

## Preprocessing Functions

In [14]:
def case_folding(text):
    # Hapus angka
    text = ''.join([char if not char.isdigit() else ' ' for char in text])
    # Hapus tanda baca (selain huruf dan spasi)
    text = ''.join([char if char.isalnum() or char.isspace() else ' ' for char in text])
    # Ubah ke huruf kecil
    text = text.lower()
    # Hapus spasi berlebih
    text = ' '.join(text.split())
    return text

def tokenization(text):
    return text.split()

def stopword_removal(tokens, stopwords):
    return [word for word in tokens if word not in stopwords]

def simple_stemming(word, kata_dasar):
    """
    Fungsi stemming sederhana untuk Bahasa Indonesia.
    Menghapus akhiran, awalan, dan sufiks hingga kata menjadi bentuk dasar.
    """
    # Cek apakah kata sudah menjadi bentuk dasar terlebih dahulu
    if word in kata_dasar:
        return word  # Jika sudah bentuk dasar, langsung kembalikan kata tersebut

    # Daftar imbuhan
    suffixes = ["lah", "kah", "ku", "mu", "nya", "kan", "an", "i"]
    prefixes = ["meng", "peng", "mem", "pem", "meny", "peny", "men", "pen", "ber", "ter", "se", "di", "ke", "per", "me", "pe"]

    # Penghapusan prefiks terlebih dahulu
    for prefix in prefixes:
        if word.startswith(prefix):
            # Mengubah prefiks sesuai aturan
            if prefix == "meng" and len(word) > 4 and word[4] in "aiueo":
                word = "k" + word[4:]
            elif prefix == "peng" and len(word) > 4 and word[4] in "aiueo":
                word = "k" + word[4:]
            elif prefix == "mem" and len(word) > 3 and word[3] in "aiueo":
                word = "p" + word[3:]
            elif prefix == "pem" and len(word) > 3 and word[3] in "aiueo":
                word = "p" + word[3:]
            elif prefix == "meny" and len(word) > 4 and word[4] in "aiueo":
                word = "s" + word[4:]
            elif prefix == "peny" and len(word) > 4 and word[4] in "aiueo":
                word = "s" + word[4:]
            elif prefix == "men" and len(word) > 3 and word[3] in "aiueo":
                word = "t" + word[3:]
            elif prefix == "pen" and len(word) > 3 and word[3] in "aiueo":
                word = "t" + word[3:]
            elif prefix == "pem" and len(word) > 3 and word[3] == "r":
                word = "p" + word[3:]
            break  # Setelah mengubah, lanjutkan ke kata lainnya
    
    # Jika tidak ada perubahan pada prefiks, lanjutkan untuk menghapus prefiks lain
    for prefix in prefixes:
        if word.startswith(prefix):
            word = word[len(prefix):]  # Hapus prefiks jika belum ada perubahan
            break  # Setelah menghapus prefiks, berhenti mencari prefiks lainnya

    # Kembali ke pengecekan kata dasar setelah penghapusan prefiks
    if word in kata_dasar:
        return word

    # Penghapusan sufiks jika kata tidak dalam kata dasar
    for suffix in suffixes:
        if word.endswith(suffix):
            word = word[:-len(suffix)]
            # Cek setelah menghapus sufiks
            if word in kata_dasar:
                return word
            break  # Setelah menghapus sufiks, berhenti mencari sufiks lainnya

    return word


def stemming(tokens, kata_dasar):
    """
    Fungsi untuk melakukan stemming pada daftar token dengan membandingkan kata dengan kata dasar.
    """
    changes = []  # Daftar untuk menyimpan perubahan kata
    stemmed_tokens = []
    for word in tokens:
        stemmed_word = simple_stemming(word, kata_dasar)
        stemmed_tokens.append(stemmed_word)
        if word != stemmed_word:  # Catat perubahan kata
            changes.append((word, stemmed_word))

    # Cetak perubahan kata jika ada
    #if changes:
        #print("Kata yang diganti:", changes)

    return stemmed_tokens


def preprocessing(text, stopwords, kata_dasar):
    if not isinstance(text, str):  # Handle non-string inputs
        text = str(text)
    text = case_folding(text)
    tokens = tokenization(text)
    tokens = stopword_removal(tokens, stopwords)
    tokens = stemming(tokens, kata_dasar)
    return tokens

# Fungsi untuk memuat daftar kata dasar dari file
def load_kata_dasar(file_path):
    with open(file_path, "r") as file:
        return set(file.read().splitlines())


# Memuat daftar kata dasar dari file
kata_dasar = load_kata_dasar("kata-dasar.txt")

# Define stopwords
# Membaca daftar stopword kustom dari file txt
file_path = r'C:\Users\Monika Hermiani\Documents\Semester 7\Information Retrieval\stopword-list.txt'
with open(file_path, "r") as file:
    stopwords_list = file.read().splitlines()  # Membaca setiap baris sebagai elemen list
# Menggabungkan daftar default dan kustom, menggunakan `set` untuk menghindari duplikasi 
stopwords = set(stopwords_list)

# Preprocess documents
processed_docs = [preprocessing(abstract, stopwords, kata_dasar) for abstract in abstracts]

# Display preprocessing results
print("Preprocessing results:")
for i, (original, processed) in enumerate(zip(abstracts[:5], processed_docs[:5])):
    print(f"\nDocument {i + 1}:")
    print(f"Original: {original}")
    print(f"Processed: {processed}")

Preprocessing results:

Document 1:
Original: Abstrak— Gambar citra digital merupakan larik (array)
yang berisi nilai kompleks dengan bit tertentu yang dapat
dihitung secara matetmatis. Pada penelitian ini ada dua metode
yang digunakan untuk membandingkan hasil akurasi klasifikasi
citra digital RGB yaitu dengan metode Naive Bayes dan K-
Nearest Neighbor (KNN). Metode klasifikasi Naive Bayes
berdasarkan perhitungan matematika probabilitas yang
sederhana dan metode klasifikasi K-Nearest Neighbor (KNN)
berdasarkan pada perhitungan kedekatan atau K. Kedua
metode tersebut diberikan data set digital citra scenes RGB
yang sama untuk proses pengelompokan dan pengklasifikasian
selanjutnya data akan dilatih dan diuji untuk mendapatkan
hasil akurasi. Berdasarkan klasifikasi data set citra digital RGB
menggunakan metode K-Nearest Neighbor memiliki tingkat
akurasi lebih besar dari metode Naive Bayes. Metode
pemrosesan data citra digital menghasilkan Naive Bayes dengan
akurasi yang diperoleh adalah 

## Build Inverted Index

In [15]:
# Membuat inverted index secara manual
def build_index_manual(documents):
    # 1. Membuat rangkaian pasangan term dan doc_id
    term_doc_pairs = []
    for doc_id, tokens in enumerate(documents):
        for token in tokens:
            term_doc_pairs.append((token, doc_id))

    # 2. Mengubah urutan berdasarkan term, kemudian berdasarkan doc_id
    term_doc_pairs = sorted(term_doc_pairs, key=lambda x: (x[0], x[1]))

    # 3. Term dalam 1 dokumen digabungkan
    index = {}
    for term, doc_id in term_doc_pairs:
        if term not in index:
            index[term] = []
        if doc_id not in index[term]:  # Pastikan tidak ada duplikasi doc_id
            index[term].append(doc_id)

    # 4. Memisahkan term ke dalam kamus dan posting list
    return index

# Membuat inverted index manual
inverted_index_manual = build_index_manual(processed_docs)

# Menampilkan inverted index
print("\nInverted Index Manual:")
for token, doc_ids in list(inverted_index_manual.items()):  # Tampilkan 10 token pertama
    print(f"Token: {token}")
    print(f"Document IDs: {', '.join(map(str, doc_ids))}\n")



Inverted Index Manual:
Token: 
Document IDs: 2, 12, 23, 89, 93, 100, 111, 142, 144, 151, 175, 219, 231, 245

Token: a
Document IDs: 170, 238, 274, 275

Token: ab
Document IDs: 274

Token: abc
Document IDs: 22

Token: abjad
Document IDs: 220

Token: abnormal
Document IDs: 151

Token: abortus
Document IDs: 141

Token: abot
Document IDs: 43

Token: absolute
Document IDs: 254

Token: abstract
Document IDs: 170

Token: abstraksi
Document IDs: 38, 74, 200, 210, 246, 252

Token: abusive
Document IDs: 237

Token: academy
Document IDs: 230

Token: acak
Document IDs: 73, 209, 220

Token: acapkal
Document IDs: 26

Token: acara
Document IDs: 103, 105, 221

Token: accuracy
Document IDs: 18, 33, 37, 55, 56, 60, 65, 69, 76, 79, 81, 84, 89, 112, 113, 114, 118, 121, 125, 137, 167, 170, 187, 199, 221, 241, 253, 295

Token: accurasy
Document IDs: 241

Token: accurate
Document IDs: 21

Token: action
Document IDs: 214

Token: activity
Document IDs: 286

Token: actual
Document IDs: 22

Token: acu
Document 

Compression Posting List

In [16]:
# Fungsi encode VB (Variable Byte Encoding)
def encode_vb(number):
    """
    Mengkodekan angka ke dalam Variable Byte Encoding.
    - Setiap byte terdiri dari 7 bit data dan 1 bit flag (MSB).
    - Byte terakhir ditandai dengan bit MSB = 1.
    """
    bytes_ = []
    while True:
        byte = number & 127  # Ambil 7 bit terakhir (mask 01111111)
        bytes_.insert(0, byte)  # Sisipkan byte di awal daftar
        if number < 128:
            break
        number >>= 7  # Geser 7 bit ke kanan
    bytes_[-1] |= 128  # Set bit MSB pada byte terakhir
    return bytes_

# Fungsi untuk mengompresi posting list dengan VB Encoding
def compress_posting_list(inverted_index):
    """
    Melakukan kompresi posting list dalam inverted index menggunakan VB Encoding.
    - Gap encoding diterapkan sebelum encoding dengan VB.
    """
    compressed_index = {}

    for term, posting_list in inverted_index.items():
        # 1. Gap encoding: Menghitung selisih antar docIDs
        gaps = [posting_list[0]] + [posting_list[i] - posting_list[i - 1] for i in range(1, len(posting_list))]
        
        # 2. Variable Byte Encoding untuk setiap gap
        compressed_bytes = []
        for gap in gaps:
            compressed_bytes.extend(encode_vb(gap))  # Encode setiap gap
        
        # 3. Simpan hasil VB Encoding ke index terkompresi
        compressed_index[term] = compressed_bytes
    
    return compressed_index

# Kompresi posting list inverted index dengan VB Encoding
compressed_index = compress_posting_list(inverted_index_manual)

# Menampilkan hasil kompresi untuk beberapa token
print("\nCompressed Posting List (with VB Encoding):")
for token, compressed_bytes in list(compressed_index.items()):  # Tampilkan semua token
    print(f"Token: {token}")
    print(f"Compressed Bytes: {compressed_bytes}\n")



Compressed Posting List (with VB Encoding):
Token: 
Compressed Bytes: [130, 138, 139, 194, 132, 135, 139, 159, 130, 135, 152, 172, 140, 142]

Token: a
Compressed Bytes: [1, 170, 196, 164, 129]

Token: ab
Compressed Bytes: [2, 146]

Token: abc
Compressed Bytes: [150]

Token: abjad
Compressed Bytes: [1, 220]

Token: abnormal
Compressed Bytes: [1, 151]

Token: abortus
Compressed Bytes: [1, 141]

Token: abot
Compressed Bytes: [171]

Token: absolute
Compressed Bytes: [1, 254]

Token: abstract
Compressed Bytes: [1, 170]

Token: abstraksi
Compressed Bytes: [166, 164, 254, 138, 164, 134]

Token: abusive
Compressed Bytes: [1, 237]

Token: academy
Compressed Bytes: [1, 230]

Token: acak
Compressed Bytes: [201, 1, 136, 139]

Token: acapkal
Compressed Bytes: [154]

Token: acara
Compressed Bytes: [231, 130, 244]

Token: accuracy
Compressed Bytes: [146, 143, 132, 146, 129, 132, 133, 132, 135, 131, 130, 131, 133, 151, 129, 129, 132, 131, 132, 140, 158, 131, 145, 140, 150, 148, 140, 170]

Token: accu

## TF-IDF Calculation

In [6]:
def calculate_tf(doc_tokens):
    tf = {}
    for token in doc_tokens:
        tf[token] = tf.get(token, 0) + 1
    for token in tf:
        tf[token] /= len(doc_tokens)
    return tf

def calculate_idf(documents):
    idf = {}
    total_docs = len(documents)
    for doc_tokens in documents:
        for token in set(doc_tokens):
            idf[token] = idf.get(token, 0) + 1
    for token in idf:
        idf[token] = log(total_docs / idf[token])
    return idf

def calculate_tfidf(documents):
    tfidf = []
    idf = calculate_idf(documents)
    for doc_tokens in documents:
        tf = calculate_tf(doc_tokens)
        tfidf.append({token: tf[token] * idf[token] for token in tf})
    return tfidf, idf

tfidf_matrix, idf = calculate_tfidf(processed_docs)
print(tfidf_matrix[:2])  # Display first 2 TF-IDF vectors

[{'gambar': 0.022873257859837695, 'citra': 0.0731263344031822, 'digital': 0.11397495827657206, 'rupa': 0.0050096434099497315, 'larik': 0.04637221524110733, 'array': 0.04637221524110733, 'isi': 0.02124178879103972, 'nilai': 0.005967229065692688, 'kompleks': 0.033287354164407326, 'bit': 0.04637221524110733, 'tentu': 0.009879236625398871, 'hitung': 0.051713744785368075, 'matetmatis': 0.04637221524110733, 'teliti': 0.0013853088286884623, 'metode': 0.02484713602751612, 'guna': 0.0004394906079336491, 'banding': 0.02413454909723731, 'hasil': 0.0027511096133010803, 'akurasi': 0.011334652730682862, 'klasifikasi': 0.013731969903835345, 'rgb': 0.1272202603391267, 'naive': 0.07896676255945685, 'bayes': 0.04631846679627499, 'k': 0.06571090965690778, 'nearest': 0.06475238253995487, 'neighbor': 0.07203076593219006, 'knn': 0.03589064899495482, 'dasar': 0.020209393937928036, 'matematika': 0.04073687230972566, 'probabilitas': 0.03510152937834399, 'sederhana': 0.028508600791219366, 'dekat': 0.01988362550

Process Query

In [7]:
# Query Processing Block
def process_query(query, stopwords):
    """
    Proses query untuk menghitung query vector berdasarkan TF-IDF,
    di mana TF dan IDF dihitung hanya berdasarkan query.
    Args:
        query (str): Query dari pengguna.
        stopwords (set): Daftar stopword.
    Returns:
        tuple: Token hasil preprocessing dan query vector (TF-IDF).
    """
    # Preprocessing query
    tokens = preprocessing(query, stopwords, kata_dasar)

    # Calculate TF (term frequency) for the query
    tf = calculate_tf(tokens)

    # Calculate IDF for the query
    total_terms = len(set(tokens))
    idf = {token: log(total_terms / tokens.count(token)) for token in set(tokens)}

    # Calculate query vector (TF-IDF)
    query_vector = {token: tf[token] * idf[token] for token in tokens}

    return tokens, query_vector


# Input query dari pengguna
query = input("Masukkan query: ")

# Process query
query_tokens, query_vector = process_query(query, stopwords)

# Display results
print("\nQuery Tokens:", query_tokens)
print("\nQuery Vector (TF-IDF):")
for term, tfidf in query_vector.items():
    print(f"  Term: {term}, TF-IDF: {tfidf:.4f}")



Query Tokens: ['analisis', 'sentimen', 'twitter']

Query Vector (TF-IDF):
  Term: analisis, TF-IDF: 0.3662
  Term: sentimen, TF-IDF: 0.3662
  Term: twitter, TF-IDF: 0.3662


Retrieve Document by Inverted Index

In [8]:
# Retrieve Documents Using Inverted Index
def retrieve_by_inverted_index(query_tokens, inverted_index_manual, titles):
    """
    Retrieve documents that match the query tokens using the inverted index.
    Args:
        query_tokens (list): Tokens dari query.
        inverted_index (dict): Inverted index.
        titles (list): Daftar judul dokumen.
    Returns:
        list: Daftar dokumen yang cocok dengan query.
    """
    matched_doc_ids = set()
    for token in query_tokens:
        if token in inverted_index_manual:
            matched_doc_ids.update(inverted_index_manual[token])
    return [(doc_id, titles[doc_id]) for doc_id in matched_doc_ids]

# Pastikan 'query_tokens' sudah dihasilkan dari sel sebelumnya
if 'query_tokens' in locals():
    # Retrieve documents using inverted index
    matched_docs = retrieve_by_inverted_index(query_tokens, inverted_index_manual, titles)

    # Display retrieved documents
    if matched_docs:
        print("\nDokumen yang Mengandung Kata Kunci Query:")
        for i, (doc_id, title) in enumerate(matched_docs, 1):
            print(f"{i}. {title}. {doc_id}")
    else:
        print("\nTidak ada dokumen yang cocok dengan query.")
else:
    print("Error: 'query_tokens' belum tersedia. Jalankan pemrosesan query terlebih dahulu.")



Dokumen yang Mengandung Kata Kunci Query:
1. Akurasi Klasifikasi Citra Digital Scenes RGB Menggunakan Model K-Nearest Neighbor dan Naive Bayes. 0
2. ALGORITMA NAÏVE BAYES DALAM ANALISIS SENTIMEN UNTUK KLASIFIKASI PADA LAYANAN INTERNET PT XYZ. 4
3. Algoritma Pengenalan Ekspresi Wajah Menggunakan Wavelet Neural Networks. 7
4. Algoritme decision tree untuk mendeteksi ujaran kebencian dan bahasa kasar multilabel pada Twitter berbahasa Indonesia. 8
5. Analisa Sentimen Opini Publik Tentang Vaksinasi Covid-19 di Indonesia Menggunakan Naïve Bayes dan Decission Tree. 14
6. Analisa Sentimen Pengguna Transportasi Jakarta Terhadap Transjakarta Menggunakan Metode Naives Bayes dan K-Nearest Neighbor. 15
7. Analisa Sentimen Pengunjung Hotel Dengan K-Nearest Neighbor Studi Kasus Hotel Pop! Surabaya. 16
8. Analisa tanggapan terhadap PSBB di indonesia dengan algoritma decision tree pada twitter. 18
9. ANALISI DAN SIMULASI PENCARIAN REFF DAN VERSE LAGU PADA MUSIK DIGITAL DENGAN METODE LINEAR PREDICTIV

Calculate Cosine Similarity

In [9]:
from math import sqrt

# Cosine Similarity Calculation
def calculate_similarity(tfidf_matrix, query_vector):
    """
    Calculate cosine similarity between documents and a query.
    Args:
        tfidf_matrix (list of dicts): TF-IDF vectors for all documents.
        query_vector (dict): TF-IDF vector for the query.
    Returns:
        list: Cosine similarity values for each document.
    """
    similarities = []
    
    for doc_id, doc_tfidf in enumerate(tfidf_matrix):
        # Menghitung dot product antara dokumen dan query
        dot_product = sum(
            doc_tfidf.get(token, 0) * query_vector.get(token, 0)
            for token in query_vector
        )
        
        # Menghitung norma vektor dokumen
        doc_norm = sqrt(sum(value**2 for value in doc_tfidf.values()))
        
        # Menghitung norma vektor query
        query_norm = sqrt(sum(value**2 for value in query_vector.values()))
        
        # Jika salah satu norma bernilai nol, similarity = 0
        if doc_norm * query_norm == 0:
            similarity = 0
        else:
            # Menghitung cosine similarity
            similarity = dot_product / (doc_norm * query_norm)
        
        # Menyimpan hasil similarity untuk dokumen ini
        similarities.append((doc_id, similarity))
    
    return similarities

# Display cosine similarity results
cosine_similarities = calculate_similarity(tfidf_matrix, query_vector)

print("\nCosine Similarity Results:")
for doc_id, similarity in cosine_similarities:
    print(f"Document {doc_id + 1}: {similarity:.4f}")



Cosine Similarity Results:
Document 1: 0.0184
Document 2: 0.0000
Document 3: 0.0000
Document 4: 0.0000
Document 5: 0.2002
Document 6: 0.0000
Document 7: 0.0000
Document 8: 0.0306
Document 9: 0.0273
Document 10: 0.0000
Document 11: 0.0000
Document 12: 0.0000
Document 13: 0.0000
Document 14: 0.0000
Document 15: 0.0676
Document 16: 0.2609
Document 17: 0.0239
Document 18: 0.0000
Document 19: 0.1080
Document 20: 0.0000
Document 21: 0.0084
Document 22: 0.0000
Document 23: 0.0000
Document 24: 0.0334
Document 25: 0.0000
Document 26: 0.0000
Document 27: 0.0369
Document 28: 0.0000
Document 29: 0.0536
Document 30: 0.0000
Document 31: 0.0000
Document 32: 0.0000
Document 33: 0.0000
Document 34: 0.0000
Document 35: 0.0000
Document 36: 0.0000
Document 37: 0.0000
Document 38: 0.0000
Document 39: 0.0000
Document 40: 0.0000
Document 41: 0.0000
Document 42: 0.0000
Document 43: 0.1030
Document 44: 0.0000
Document 45: 0.0109
Document 46: 0.0000
Document 47: 0.0000
Document 48: 0.1081
Document 49: 0.1008
D

Rank Document using Cosine Similarity

In [10]:
from math import sqrt

# Rank documents using cosine similarity
def rank_documents(tfidf_matrix, query_vector, matched_docs, titles):
    """
    Rank documents based on cosine similarity with the query.
    
    Args:
        tfidf_matrix (list of dicts): TF-IDF vectors for all documents.
        query_vector (dict): TF-IDF vector for the query.
        matched_docs (list of tuples): List of matched document IDs and titles.
        titles (list): List of document titles.
    
    Returns:
        list: Ranked documents with similarity scores.
    """
    ranked_results = []
    for doc_id, _ in matched_docs:
        # Calculate cosine similarity
        dot_product = sum(
            tfidf_matrix[doc_id].get(token, 0) * query_vector.get(token, 0)
            for token in query_vector
        )
        doc_norm = sqrt(sum(value**2 for value in tfidf_matrix[doc_id].values()))
        query_norm = sqrt(sum(value**2 for value in query_vector.values()))
        if doc_norm * query_norm == 0:
            similarity = 0
        else:
            similarity = dot_product / (doc_norm * query_norm)
        
        # Append document ID and similarity score
        ranked_results.append((doc_id, similarity))

    # Sort results by similarity score in descending order
    ranked_results.sort(key=lambda x: x[1], reverse=True)
    return ranked_results

# Calculate rankings for matched documents
ranked_results = rank_documents(tfidf_matrix, query_vector, matched_docs, titles)

# Display ranked results
print("\nDokumen yang Diranking Berdasarkan Cosine Similarity:")
for rank, (doc_id, similarity) in enumerate(ranked_results, start=1):
    print(f"{rank}. {titles[doc_id]} (Relevansi: {similarity:.4f})")



Dokumen yang Diranking Berdasarkan Cosine Similarity:
1. ANALISIS SENTIMEN JASA EKSPEDISI BARANG MENGGUNAKAN METODE NAÏVE BAYES (Relevansi: 0.2997)
2. Analisa Sentimen Pengguna Transportasi Jakarta Terhadap Transjakarta Menggunakan Metode Naives Bayes dan K-Nearest Neighbor (Relevansi: 0.2609)
3. ANALISIS SENTIMEN TWITTER MENGGUNAKAN TEXT MINING DENGAN ALGORITMA NAÏVE BAYES CLASSIFIER (Relevansi: 0.2543)
4. ANALISIS SENTIMEN TERHADAP PEMERINTAHAN JOKO WIDODO PADA MEDIA SOSIAL TWITTER MENGGUNAKAN ALGORITMA NAIVES BAYES CLASSIFIER (Relevansi: 0.2532)
5. Analisis Sentimen Terhadap Layanan Indihome Berdasarkan Twitter Dengan Metode Klasifikasi Support Vector Machine (Relevansi: 0.2503)
6. ANALISIS SENTIMEN TERHADAP PELAYANAN PT PLN DI JAKARTA PADA TWITTER DENGAN ALGORITMA K-NEAREST NEIGHBOR (Relevansi: 0.2410)
7. ANALISIS SENTIMEN WACANA PEMINDAHAN IBU KOTA INDONESIA MENGGUNAKAN ALGORITMA SUPPORT VECTOR MACHINE (SVM) (Relevansi: 0.2276)
8. ANALISIS SENTIMEN PELANGGAN TOKO ONLINE JD.ID M

Find Relevan Document by Titles

In [11]:
# Cari dokumen yang relevan berdasarkan judul dan abstrak
def find_relevant_documents(query, titles, abstracts):
    """
    Cari dokumen yang relevan dengan query berdasarkan judul dan abstrak.

    Args:
        query (str): Query input dari pengguna.
        titles (list of str): Daftar judul dokumen.
        abstracts (list of str): Daftar abstrak dokumen.

    Returns:
        list of tuple: Daftar dokumen relevan dalam bentuk (judul, abstrak).
    """
    # Preprocess query menjadi token untuk pencocokan kata
    query_tokens = set(query.lower().split())

    # Cari dokumen yang relevan berdasarkan judul terlebih dahulu
    relevant_docs_by_title = []
    for title, abstract in zip(titles, abstracts):
        title_tokens = set(title.lower().split())
        #mencocokkan token query dengan token yang ada pada judul dokumen
        if query_tokens & title_tokens:
            relevant_docs_by_title.append((title, abstract))

    # Jika tidak ada dokumen relevan berdasarkan judul, cari berdasarkan abstrak
    if relevant_docs_by_title:
        return relevant_docs_by_title

    relevant_docs_by_abstract = []
    for title, abstract in zip(titles, abstracts):
        abstract_tokens = set(abstract.lower().split())
        if query_tokens & abstract_tokens:
            relevant_docs_by_abstract.append((title, abstract))

    return relevant_docs_by_abstract

# Cari dokumen relevan
relevant_docs = find_relevant_documents(query, titles, abstracts)

# Tampilkan hasil
if not relevant_docs:
    print("Tidak ditemukan dokumen relevan berdasarkan query.")
else:
    print("\nDokumen yang Relevan Berdasarkan Judul dan Abstrak:")
    for i, (title, abstract) in enumerate(relevant_docs, start=1):
        print(f"{i}. Judul: {title}")
        #print(f"   Abstrak: {abstract}\n")



Dokumen yang Relevan Berdasarkan Judul dan Abstrak:
1. Judul: ALGORITMA NAÏVE BAYES DALAM ANALISIS SENTIMEN UNTUK KLASIFIKASI PADA LAYANAN INTERNET PT XYZ
2. Judul: Algoritme decision tree untuk mendeteksi ujaran kebencian dan bahasa kasar multilabel pada Twitter berbahasa Indonesia
3. Judul: Analisa Sentimen Opini Publik Tentang Vaksinasi Covid-19 di Indonesia Menggunakan Naïve Bayes dan Decission Tree
4. Judul: Analisa Sentimen Pengguna Transportasi Jakarta Terhadap Transjakarta Menggunakan Metode Naives Bayes dan K-Nearest Neighbor
5. Judul: Analisa Sentimen Pengunjung Hotel Dengan K-Nearest Neighbor Studi Kasus Hotel Pop! Surabaya
6. Judul: Analisa tanggapan terhadap PSBB di indonesia dengan algoritma decision tree pada twitter
7. Judul: ANALISIS ACCURATE LEARNING BACKPROPAGATION NEURAL NETWORK PADA PENGENALAN WAJAH
8. Judul: Analisis Arsitektur Jaringan Syaraf Tiruan Untuk Peramalan Penjualan Air Minum Dalam Kemasan
9. Judul: Analisis Dan Implementasi Support Vector Machine Den

Evaluate Precision and Recall

In [12]:
# Evaluate Precision and Recall
def evaluate_retrieval(ranked_results, relevant_docs, titles):
    """
    Evaluasi precision dan recall dari hasil pencarian dokumen.

    Args:
        ranked_results (list of tuple): Hasil ranking dokumen (doc_id, similarity).
        relevant_docs (list of tuple): Daftar dokumen relevan (judul, abstrak).
        titles (list of str): Daftar semua judul dokumen.

    Returns:
        tuple: Precision dan recall dari hasil pencarian.
    """
    # Ekstrak judul dari dokumen relevan
    relevant_titles = [doc[0] for doc in relevant_docs]

    # Judul dokumen yang berhasil diambil
    retrieved_titles = [titles[doc_id] for doc_id, _ in ranked_results]

    # Dokumen relevan yang berhasil diambil
    relevant_retrieved = [
        title for title in retrieved_titles if title in relevant_titles
    ]

    # Precision: Proporsi dokumen relevan dari dokumen yang diambil
    precision = (
        len(relevant_retrieved) / len(retrieved_titles) if retrieved_titles else 0
    )

    # Recall: Proporsi dokumen relevan yang berhasil diambil dari total dokumen relevan
    recall = len(relevant_retrieved) / len(relevant_titles) if relevant_titles else 0

    return precision, recall


# Cari dokumen relevan (judul dan abstrak)
relevant_docs = find_relevant_documents(query, titles, abstracts)

# Evaluasi precision dan recall
if not relevant_docs:
    print("Tidak ditemukan dokumen relevan berdasarkan query.")
else:
    precision, recall = evaluate_retrieval(ranked_results, relevant_docs, titles)

    # Tampilkan hasil evaluasi
    print("\nPrecision: {:.4f}".format(precision))
    print("Recall: {:.4f}".format(recall))



Precision: 0.8699
Recall: 0.7985


In [13]:
import sys

# Function to calculate memory usage of a dictionary
def calculate_memory_usage(dictionary):
    total_size = sys.getsizeof(dictionary)  # Size of the dictionary object
    for key, value in dictionary.items():
        total_size += sys.getsizeof(key)  # Size of each key
        total_size += sys.getsizeof(value)  # Size of each value
    return total_size

# Calculate memory usage for the dictionary (entire inverted index structure)
inverted_index_memory = calculate_memory_usage(inverted_index_manual)

# Calculate memory usage specifically for posting lists (values in the dictionary)
posting_list_memory = sum(sys.getsizeof(postings) for postings in inverted_index_manual.values())

# Memory usage for the dictionary keys only
dictionary_keys_memory = sum(sys.getsizeof(key) for key in inverted_index_manual.keys())

# Memory usage for the entire inverted index structure (including keys and posting lists)
total_memory_usage = inverted_index_memory

print(f"Memory usage of inverted index structure (dictionary): {inverted_index_memory / 1024:.2f} KB")
print(f"Memory usage of dictionary keys: {dictionary_keys_memory / 1024:.2f} KB")
print(f"Memory usage of posting lists: {posting_list_memory / 1024:.2f} KB")
print(f"Total memory usage (dictionary + posting lists): {total_memory_usage / 1024:.2f} KB")


Memory usage of inverted index structure (dictionary): 782.31 KB
Memory usage of dictionary keys: 193.33 KB
Memory usage of posting lists: 444.88 KB
Total memory usage (dictionary + posting lists): 782.31 KB
